# Static Model Test
This notebook tests the defined models using a gradio inteface where one can simply upload an image or video, choose a model and then get a classification of the perfromed exercise.

In [1]:
import gradio as gr
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from mediapipe.framework.formats import landmark_pb2

c:\Users\hannes\01_Code\02_master_rci\03_Semester_PoliMi\02_nearables_lab\gym_tracking\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

MODEL_PATH = "..\skeleton_lstm_multiclass3.h5"
# bench_press: 0, bulgarian_squat: 1, lat_machine: 2, pull_up: 3, push_up: 4, split_squat: 5
CLASSES = ["pull_up", "push_up", "split_squat"]

KEYPOINT_DIM = 132  # 33 landmarks with x,y,z,visibility

# ——— load trained model ———
model = tf.keras.models.load_model(MODEL_PATH)

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\hannes\AppData\Local\Temp\ipykernel_2672\538120499.py:3: SyntaxWarning: invalid escape sequence '\s'
  MODEL_PATH = "..\skeleton_lstm_multiclass3.h5"


In [3]:
# ——— init Mediapipe Pose & etc. ———
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

pose = mp_pose.Pose(
    static_image_mode=True,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

In [4]:
# ——— helper to extract keypoints array from Mediapipe results ———
def extract_keypoints_from_results(results):
    # build list in the same order as training: (lm0.x, lm0.y, lm0.z, lm0.v, lm1.x, …)
    kpts = []
    for lm in results.pose_landmarks.landmark:
        kpts.extend([lm.x, lm.y, lm.z, lm.visibility])
    return np.array(kpts, dtype=np.float32)  # shape = (132,)

In [5]:
# FLIPPED
def process_video_flipped(video):
    # Initialize video capture
    cap = cv2.VideoCapture(video)
    sequence_flipped = []  # Only store flipped keypoints
    frame_count = 0
    max_frames = 30
    
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Convert frame to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Get pose landmarks
        results = pose.process(frame_rgb)
        
        if results.pose_landmarks:
            # Extract keypoints
            keypoints = extract_keypoints_from_results(results)
            
            # Create flipped keypoints by negating x coordinates
            keypoints_flipped = keypoints.copy()
            for i in range(0, len(keypoints_flipped), 4):  # Step by 4 because each point has x,y,z,visibility
                keypoints_flipped[i] = 1-keypoints_flipped[i]  # Flip x coordinate
            
            sequence_flipped.append(keypoints_flipped)
            frame_count += 1
    
    cap.release()
    
    # Pad sequence if needed
    if len(sequence_flipped) < max_frames:
        last_frame = sequence_flipped[-1] if sequence_flipped else np.zeros(KEYPOINT_DIM)
        while len(sequence_flipped) < max_frames:
            sequence_flipped.append(last_frame)
    
    # Convert sequence to numpy array and reshape for model input
    sequence_flipped = np.array(sequence_flipped).reshape(1, max_frames, KEYPOINT_DIM)
    
    # Get model predictions
    predictions = model.predict(sequence_flipped, verbose=0)[0]
    
    # Get top 3 predictions
    top_3_idx = np.argsort(predictions)[-3:][::-1]
    top_3_classes = [CLASSES[i] for i in top_3_idx]
    top_3_confidences = [float(predictions[i]) for i in top_3_idx]
    
    # Create prediction text
    prediction_text = "Top 3 Predictions (Flipped):\n"
    for i in range(3):
        prediction_text += f"{i+1}. {top_3_classes[i]}: {top_3_confidences[i]:.2%}\n"
    
    # Create a visualization of the flipped pose
    if len(sequence_flipped) > 0:
        # Create a blank image
        last_frame = np.zeros((480, 640, 3), dtype=np.uint8)
        
        # Create a new NormalizedLandmarkList for the flipped pose
        flipped_landmarks = landmark_pb2.NormalizedLandmarkList()
        
        # Get the last keypoints from the sequence
        last_keypoints = sequence_flipped[0, -1]  # Shape: (132,)
        
        # Convert keypoints back to landmark format
        for i in range(0, len(last_keypoints), 4):
            landmark = flipped_landmarks.landmark.add()
            landmark.x = last_keypoints[i]
            landmark.y = last_keypoints[i + 1]
            landmark.z = last_keypoints[i + 2]
            landmark.visibility = last_keypoints[i + 3]
        
        # Draw flipped pose landmarks
        mp_drawing.draw_landmarks(
            last_frame,
            flipped_landmarks,
            mp_pose.POSE_CONNECTIONS
        )
        
        # Add prediction text to frame
        y_position = 30
        for i in range(3):
            text = f"{top_3_classes[i]}: {top_3_confidences[i]:.2%}"
            cv2.putText(last_frame, text, (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            y_position += 40
    else:
        last_frame = np.zeros((480, 640, 3), dtype=np.uint8)
        cv2.putText(last_frame, "No pose detected in video", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    return prediction_text, last_frame

In [6]:
# Create Gradio interface for flipped version
demo_flipped = gr.Interface(
    fn=process_video_flipped,
    inputs=gr.Video(),
    outputs=[
        gr.Textbox(label="Predictions (Flipped)", lines=4),
        gr.Image(label="Flipped Pose Visualization")
    ],
    title="Exercise Classification (Flipped)",
    description="Upload a video to classify the exercise using flipped pose keypoints."
)

# Launch the interface
demo_flipped.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


In [7]:
# ORIGINAL
def process_video(video):
    # Initialize video capture
    cap = cv2.VideoCapture(video)
    sequence = []
    frame_count = 0
    max_frames = 30  # We want 30 frames for the sequence
    
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Convert frame to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Get pose landmarks
        results = pose.process(frame_rgb)
        
        if results.pose_landmarks:
            # Extract keypoints
            keypoints = extract_keypoints_from_results(results)
            sequence.append(keypoints)
            frame_count += 1
    
    cap.release()
    
    # If we don't have enough frames, pad the sequence
    if len(sequence) < max_frames:
        # Pad with the last frame's keypoints
        last_frame = sequence[-1] if sequence else np.zeros(KEYPOINT_DIM)
        while len(sequence) < max_frames:
            sequence.append(last_frame)
    
    # Convert sequence to numpy array and reshape for model input
    sequence = np.array(sequence)
    sequence = sequence.reshape(1, max_frames, KEYPOINT_DIM)
    
    # Get model predictions
    predictions = model.predict(sequence, verbose=0)[0]
    
    # Get top 3 predictions
    top_3_idx = np.argsort(predictions)[-3:][::-1]
    top_3_classes = [CLASSES[i] for i in top_3_idx]
    top_3_confidences = [float(predictions[i]) for i in top_3_idx]
    
    # Create prediction text
    prediction_text = "Top 3 Predictions:\n"
    for i in range(3):
        prediction_text += f"{i+1}. {top_3_classes[i]}: {top_3_confidences[i]:.2%}\n"
    
    # Create a visualization of the last processed frame
    if len(sequence) > 0:
        last_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Draw pose landmarks
        mp_drawing.draw_landmarks(
            last_frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style()
        )
        
        # Add prediction text to frame
        y_position = 30
        for i in range(3):
            text = f"{top_3_classes[i]}: {top_3_confidences[i]:.2%}"
            cv2.putText(last_frame, text, (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            y_position += 40
    else:
        last_frame = np.zeros((480, 640, 3), dtype=np.uint8)
        cv2.putText(last_frame, "No pose detected in video", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    return prediction_text, last_frame

In [8]:
# Create Gradio interface
demo = gr.Interface(
    fn=process_video,
    inputs=gr.Video(),  # Changed from Image to Video
    outputs=[
        gr.Textbox(label="Predictions", lines=4),
        gr.Image(label="Last Frame with Pose")
    ],
    title="Exercise Classification (Video)",
    description="Upload a video to classify the exercise being performed."
)

# Launch the interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


## Results 16.06.2025

### Findings 
- **Point-of-View / Flipping Effect**: <br> when flipping the keypoints horizontally (i.e. mirroring left <-> right), the model changes its prediction for the same original video. Strongly suggests the model is “looking” at absolute x-positions in the frame (e.g. left-side bias), rather than the relative geometry of the joints.
- **Overfitting** <br> model performance drops on flipped (but otherwise identical) inputs, it’s likely memorizing frame-specific patterns (camera offset, background context) rather than learning the invariant shape of the exercise. 
- **Class imbalance** <br> between the processed csv files; low-sample classes vulnerable to misclassification:
    - bench_press: 387 elements
    - bulgarian_squat: 452 elements
    - lat_machine: 333 elements
    - pull_up: 167 elements
    - push_up: 231 elements
    - split_squat: 665 elements
- **Misclassification Bias** <br> a lot of times for 3 class classification push up or pull up are misclassified to split_squat -> bias towards overrepresented class; common for under-represented classes to be “sucked into” an overrepresented neighbor

### Mitigation Steps
- **Data Augmentation**:
    - flip/rotate all the input videos by a certain margin
    - add them twice: once normal & once vertical flipped (remember to swap left/right landmark indices (e.g. left_elbow <-> right_elbow) before training)
- **Expand Dataset**:
    - More videos from different angles, lighting, and subjects will improve generalization.
    - one may have to abandon and replace given classes such as split_squat & bulgarian_squat as they do not exist in big public datasets
- **Mediapipe** <br> normalized coordinated at top left af image, all coordinates are between [0, 1]. one could consider some improvements:
    - **Center on a root joint** <br> Subtract the hip midpoint (or another stable landmark) from every x,y,z so that your model sees poses relative to the body, not the image
    - **Scale Normalization** <br> Divide distances by the length of the torso or shoulders width so that “big” vs. “small” people don’t confuse the network
    - **Hand-crafted Features** <br> calculate distances and angles and add as extra input to the model to give it more information